## News Sentiment

# Manage API Key & Use NewsApiClient
I've saved my API key in a config file, I've loaded the file and saved my key to a variable.

In [3]:
import config

APIKEY=config.APIKEY

from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key=APIKEY)

# What are the available sources?
I've created a loop to gather all the available sources and save into a dataframe. For clarity sake I've printed the available sources as well as the categories.

In [50]:
import pandas as pd
sources = newsapi.get_sources()
print('How many total sources are available?:',len(sources['sources']))
total_sources = len(sources['sources'])
sources['sources'][0]

id=pd.Series(name='id')
name=pd.Series(name='name')
description=pd.Series(name='description')
url=pd.Series(name='url')
category=pd.Series(name='category')
language=pd.Series(name='language')
country=pd.Series(name='country')
#column_names = ['id', 'name', 'description', 'url', 'category', 'language', 'country']

for idx,outlet in enumerate(sources['sources']):
    id[idx]=outlet['id']
    name[idx] = outlet['name']
    description[idx] = outlet['description']
    url[idx] = outlet['url']
    category[idx] = outlet['category']
    language[idx] = outlet['language']
    country[idx] = outlet['country']
sourcedf=pd.concat([id,name,description,url,category,language,country],axis=1)

print('What are the categories? ', sourcedf['category'].unique())

How many total sources are available?: 128
What are the categories?  ['general' 'business' 'technology' 'sports' 'entertainment' 'health'
 'science']


# Filter the available sources
Requirements: English Language, US Country

In [49]:
sourcedf= sourcedf[(sourcedf['language']=='en') & (sourcedf['country']=='us')]
sourcedf.head()

,id,name,description,url,category,language,country
0,abc-news,ABC News,"Your trusted source for breaking news, analysi...",https://abcnews.go.com,general,en,us
3,al-jazeera-english,Al Jazeera English,"News, analysis from the Middle East and worldw...",http://www.aljazeera.com,general,en,us
6,ars-technica,Ars Technica,The PC enthusiast's resource. Power users and ...,http://arstechnica.com,technology,en,us
8,associated-press,Associated Press,The AP delivers in-depth coverage on the inter...,https://apnews.com/,general,en,us
10,axios,Axios,Axios are a new media company delivering vital...,https://www.axios.com,general,en,us


# What are the business sources?
First we isolate the business sources based on the category variable.
I've displayed the number of business sources as a sanity check, as well as tested that the indexing was working correctly.

In [44]:
business_sources = sourcedf[sourcedf['category']=='business']
business_sources = business_sources.reset_index()
display(business_sources)
print('Number of sources: ',len(business_sources))
business_sources['id'][0]

,index,id,name,description,url,category,language,country
0,16,bloomberg,Bloomberg,"Bloomberg delivers business and markets news, ...",http://www.bloomberg.com,business,en,us
1,18,business-insider,Business Insider,Business Insider is a fast-growing business si...,http://www.businessinsider.com,business,en,us
2,36,fortune,Fortune,Fortune 500 Daily and Breaking Business News,http://fortune.com,business,en,us
3,117,the-wall-street-journal,The Wall Street Journal,WSJ online coverage of breaking news and curre...,http://www.wsj.com,business,en,us


Number of sources:  4


'bloomberg'

In [20]:
# /v2/top-headlines
top_headlines = newsapi.get_everything(q='apple',
                                          sources='fortune, the-wall-street-journal, business-insider, bloomberg',
                                          from_param='2023-08-09',
                                          )
top_headlines['articles'][0]

{'status': 'ok',
 'totalResults': 185,
 'articles': [{'source': {'id': 'business-insider',
    'name': 'Business Insider'},
   'author': 'Lakshmi Varanasi',
   'title': "Apple is reportedly working on the Watch's biggest redesign yet: the 'Watch X'",
   'description': "Apple is planning to release a new model of the Apple Watch for the device's 10-year anniversary in 2024 or 2025, Bloomberg reported.",
   'url': 'https://www.businessinsider.com/apple-working-on-major-redesign-of-apple-watch-watch-x-2023-8',
   'urlToImage': 'https://i.insider.com/64da492d1f51cc001968f3ca?width=1200&format=jpeg',
   'publishedAt': '2023-08-14T16:03:56Z',
   'content': "Apple has a major redesign of the Apple Watch in the works called Watch X. (Picture above is the Series 8).Amazon\r\n<ul>\n<li>Apple is planning to release a new model of the Apple Watch for the device'… [+3023 chars]"},
  {'source': {'id': 'business-insider', 'name': 'Business Insider'},
   'author': 'htan@insider.com (Huileng Tan)',
   

# Loop through all articles 
There are two layers to this loop, the outer layer loops through the number of business sources (4), while the inner loop gets all the articles from that business source and collects the data.


In [77]:
# Init Variables
Source=pd.Series(name='source')
author=pd.Series(name='author')
title=pd.Series(name='title')
description=pd.Series(name='description')
url=pd.Series(name='url')
publishedAt=pd.Series(name='publishedAt')
content=pd.Series(name='content')
# Use Counter
counter=0
# Loop through each source 
for source in business_sources['id']:
    news_articles = newsapi.get_everything(q='apple',
                                          sources=source,
                                          from_param='2023-08-09',
                                          )
    # Gather data from each article
    for idx, article in enumerate(news_articles['articles']):
        Source[counter] = article['source']['id']
        author[counter] = article['author']
        title[counter] = article['title']
        description[counter] = article['description']
        url[counter] = article['url']
        publishedAt[counter] = article['publishedAt']
        content[counter] = article['content']
        #author=pd.concat([Source,article['author']],axis=1)
        #for details in article:
        #    print(details['title'])
        counter = counter + 1
# Combine 
all_articles = pd.concat([Source, author, title, description, url, publishedAt, content],axis=1)
all_articles.head()

,source,author,title,description,url,publishedAt,content
0,business-insider,Lakshmi Varanasi,Apple is reportedly working on the Watch's big...,Apple is planning to release a new model of th...,https://www.businessinsider.com/apple-working-...,2023-08-14T16:03:56Z,Apple has a major redesign of the Apple Watch ...
1,business-insider,htan@insider.com (Huileng Tan),The billionaire founder of a key Apple supplie...,He said no foreign investor would dare invest ...,https://www.businessinsider.com/foxconn-terry-...,2023-08-28T10:21:20Z,"Terry Gou, the billionaire founder of Foxconn ..."
2,business-insider,Business Insider,Big Tech salaries revealed: This is what devel...,Big tech salaries unveil earnings of engineers...,https://www.businessinsider.com/big-tech-salar...,2023-08-23T16:47:20Z,Business Insider analyzed salary data for work...
3,business-insider,Anastasia Chernikova,3 millennials who pivoted from 6-figure corpor...,These millennials have left jobs at places lik...,https://www.businessinsider.com/millennials-le...,2023-08-09T09:58:41Z,"Left to right: Paige Webster, Adam Parmer, and..."
4,business-insider,Zahra Tayeb,"Apple, Microsoft, Tesla, and Meta see a combin...","Four Big Tech companies - Apple, Microsoft, Te...",https://markets.businessinsider.com/news/stock...,2023-08-26T10:08:01Z,(Photo by Scott Heins/Getty Images)\r\n<ul>\n<...


# Download Stock Data

In [ ]:
import yfinance as yf
